# MNIST BDP V2

Now that we have the first version done, we are going to enrich our components even further. First, we load the previous JSON.

In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.load_from_json("../JSON/Supervised Learning V1.json")
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning']
Wires: ['W1', 'W2', 'W3', 'W4']
Systems: ['MNIST Experiment System'] > >


## Load MNIST

Currently "Load MNIST" is a primitive processor, but we are going to expand it to be a composite processor entailing a few specific things:

1. Loading the dataset
2. Conducting a test-train split
3. Doing image normalization for training data
4. Doing image normalization for testing data

In [2]:
# Add spaces
project.add_space(id="X",
                  name="X",
                  description="X Data")

project.add_space(id="Y",
                  name="Y",
                  description="Y Data")

# Add blocks
project.add_block(id="Load Supervised Dataset",
                  name="Load Supervised Dataset",
                  description="Loads the data for a supervised learning problem",
                  domain=[],
                  codomain=["X", "Y"])

project.add_block(id="Cross Validation Split",
                  name="Cross Validation Split",
                  description="Splits data into training and testing sets",
                  domain=["X", "Y"],
                  codomain=["X Train", "Y Train", "X Test", "Y Test"])

project.add_block(id="Training Data Preprocessing",
                  name="Training Data Preprocessing",
                  description="Preprocesses training data for model training",
                  domain=["X Train", "Y Train"],
                  codomain=["X Train", "Y Train"])

project.add_block(id="Testing Data Preprocessing",
                  name="Testing Data Preprocessing",
                  description="Preprocesses testing data for model evaluation, optionally can use the training data in addition",
                  domain=["X Train", "Y Train", "X Test", "Y Test"],
                  codomain=["X Test", "Y Test"])

# Add processors
project.add_processor(id="Load MNIST Dataset",
                      name="Load MNIST Dataset",
                      description="Loads the MNIST dataset",
                      parent_id="Load Supervised Dataset")

project.add_processor(id="Test-Train Split",
                      name="Test-Train Split",
                      description="Splits data into one training and one test set",
                      parent_id="Cross Validation Split")

project.add_processor(id="Image Normalization Preprocessing - Training",
                      name="Image Normalization Preprocessing - Training",
                      description="Preprocesses image-based training data",
                      parent_id="Training Data Preprocessing")

project.add_processor(id="Image Normalization Preprocessing - Testing",
                      name="Image Normalization Preprocessing - Testing",
                      description="Preprocesses testing data for model evaluation",
                      parent_id="Testing Data Preprocessing",)

### Wires & System

Similar to before, we find the wires, then build and attach the system. For the wires, we end up using all the possible wires because it is all of the ones we need and nothing more.

In [3]:
for p1, p2 in zip(["Load MNIST Dataset", "Test-Train Split", "Test-Train Split"], ["Test-Train Split", "Image Normalization Preprocessing - Training", "Image Normalization Preprocessing - Testing"]):
    print("Potential wires between processors: {} and {}".format(p1, p2))
    pprint(project.processors_map[p1].find_potential_wires(project.processors_map[p2]))
    print()
    print()

Potential wires between processors: Load MNIST Dataset and Test-Train Split
{'Ports': [],
 'Terminals': [{'Parent': 'X',
                'Source': {'Index': 0, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 0, 'Processor': 'Test-Train Split'}},
               {'Parent': 'Y',
                'Source': {'Index': 1, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 1, 'Processor': 'Test-Train Split'}}]}


Potential wires between processors: Test-Train Split and Image Normalization Preprocessing - Training
{'Ports': [],
 'Terminals': [{'Parent': 'X Train',
                'Source': {'Index': 0, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 0,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Training'}},
               {'Parent': 'Y Train',
                'Source': {'Index': 1, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 1,


In [4]:
project.add_wires([{'Parent': 'X',
                'Source': {'Index': 0, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 0, 'Processor': 'Test-Train Split'}},
               {'Parent': 'Y',
                'Source': {'Index': 1, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 1, 'Processor': 'Test-Train Split'}},
                {'Parent': 'X Train',
                'Source': {'Index': 0, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 0,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Training'}},
               {'Parent': 'Y Train',
                'Source': {'Index': 1, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 1,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Training'}},
                {'Parent': 'X Train',
                'Source': {'Index': 0, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 0,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}},
               {'Parent': 'Y Train',
                'Source': {'Index': 1, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 1,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}},
               {'Parent': 'X Test',
                'Source': {'Index': 2, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 2,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}},
               {'Parent': 'Y Test',
                'Source': {'Index': 3, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 3,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}}],
                auto_increment=True)

pprint(project.wires)

[< Wire ID: W1 Space: X Train Source: (Load MNIST, 0) Target: (Default Supervised Learning, 1) >,
 < Wire ID: W2 Space: Y Train Source: (Load MNIST, 1) Target: (Default Supervised Learning, 2) >,
 < Wire ID: W3 Space: X Test Source: (Load MNIST, 2) Target: (Default Supervised Learning, 3) >,
 < Wire ID: W4 Space: Y Test Source: (Load MNIST, 3) Target: (Default Supervised Learning, 4) >,
 < Wire ID: W5 Space: X Source: (Load MNIST Dataset, 0) Target: (Test-Train Split, 0) >,
 < Wire ID: W6 Space: Y Source: (Load MNIST Dataset, 1) Target: (Test-Train Split, 1) >,
 < Wire ID: W7 Space: X Train Source: (Test-Train Split, 0) Target: (Image Normalization Preprocessing - Training, 0) >,
 < Wire ID: W8 Space: Y Train Source: (Test-Train Split, 1) Target: (Image Normalization Preprocessing - Training, 1) >,
 < Wire ID: W9 Space: X Train Source: (Test-Train Split, 0) Target: (Image Normalization Preprocessing - Testing, 0) >,
 < Wire ID: W10 Space: Y Train Source: (Test-Train Split, 1) Target: (

### Add System

In [5]:
project.add_system(id="Load MNIST System",
                   name="Load MNIST System",
                   processors=['Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing'],
                   wires=["W5", "W6", "W7", "W8", "W9", "W10", "W11", "W12"],
                     description="The system representing loading the MNIST dataset, preparing the cross validation split, and pre-processing the data",)
project.systems_map["Load MNIST System"].display_mermaid_graphic()

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[Load MNIST System]
subgraph G0[Load MNIST Dataset - Load Supervised Dataset Block]
direction LR
X0[Load MNIST Dataset]
subgraph G0P[Ports]
direction TB
end
subgraph G0T[Terminals]
direction TB
XX0T0[X]
XX0T1[Y]
end
X0 o--o XX0T0[X]
X0 o--o XX0T1[Y]
end
subgraph G1[Test-Train Split - Cross Validation Split Block]
direction LR
X1[Test-Train Split]
subgraph G1P[Ports]
direction TB
XX1P0[X]
XX1P1[Y]
end
XX1P0[X] o--o X1
XX1P1[Y] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[X Train]
XX1T1[Y Train]
XX1T2[X Test]
XX1T3[Y Test]
end
X1 o--o XX1T0[X Train]
X1 o--o XX1T1[Y Train]
X1 o--o XX1T2[X Test]
X1 o--o XX1T3[Y Test]
end
subgraph G2[Image Normalization Preprocessing - Training - Training Data Preprocessing Block]
direction LR
X2[Image Normalization Preprocessing - Training]
subgraph G2P[Ports]
direction TB
XX2P0[X Train]
XX2P1[Y Train]
end
XX2P0[X Train] o--o X2
XX2P1[Y Train] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[X Train]
XX2T1[Y Train]
end
X2 o--o XX2T0[X Train]
X2 o--o XX2T1[Y Train]
end
subgraph G3[Image Normalization Preprocessing - Testing - Testing Data Preprocessing Block]
direction LR
X3[Image Normalization Preprocessing - Testing]
subgraph G3P[Ports]
direction TB
XX3P0[X Train]
XX3P1[Y Train]
XX3P2[X Test]
XX3P3[Y Test]
end
XX3P0[X Train] o--o X3
XX3P1[Y Train] o--o X3
XX3P2[X Test] o--o X3
XX3P3[Y Test] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[X Test]
XX3T1[Y Test]
end
X3 o--o XX3T0[X Test]
X3 o--o XX3T1[Y Test]
end
XX0T0[X] ---> XX1P0[X]
XX0T1[Y] ---> XX1P1[Y]
XX1T0[X Train] ---> XX2P0[X Train]
XX1T1[Y Train] ---> XX2P1[Y Train]
XX1T0[X Train] ---> XX3P0[X Train]
XX1T1[Y Train] ---> XX3P1[Y Train]
XX1T2[X Test] ---> XX3P2[X Test]
XX1T3[Y Test] ---> XX3P3[Y Test]
end

```

### Attatch System

When we find potential mappings, we can see a few cases where there are multiple options. In all cases we want the second version where an image normalization produces the space (instead of pre-normalization versions of test-train split).

In [9]:
possible = project.processors_map["Load MNIST"].find_potential_subsystems_mappings(project.systems_map["Load MNIST System"])
pprint(possible)

{'Port Mappings': [],
 'Terminal Mappings': [[{'Index': 0, 'Processor': 'Test-Train Split'},
                        {'Index': 0,
                         'Processor': 'Image Normalization Preprocessing - '
                                      'Training'}],
                       [{'Index': 1, 'Processor': 'Test-Train Split'},
                        {'Index': 1,
                         'Processor': 'Image Normalization Preprocessing - '
                                      'Training'}],
                       [{'Index': 2, 'Processor': 'Test-Train Split'},
                        {'Index': 0,
                         'Processor': 'Image Normalization Preprocessing - '
                                      'Testing'}],
                       [{'Index': 3, 'Processor': 'Test-Train Split'},
                        {'Index': 1,
                         'Processor': 'Image Normalization Preprocessing - '
                                      'Testing'}]]}


In [14]:
port_mappings = []
terminal_mappings = [x[1] for x in possible["Terminal Mappings"]]
print("Terminal Mappings: ", terminal_mappings)

project.attach_subsystem(project.processors_map["Load MNIST"],
                          project.systems_map["Load MNIST System"],
                          port_mappings,
                          terminal_mappings)

# Display the processor with and without the subsystem
project.processors_map["Load MNIST"].display_mermaid_graphic()
project.processors_map["Load MNIST"].display_mermaid_graphic(composite=True)

Terminal Mappings:  [{'Processor': 'Image Normalization Preprocessing - Training', 'Index': 0}, {'Processor': 'Image Normalization Preprocessing - Training', 'Index': 1}, {'Processor': 'Image Normalization Preprocessing - Testing', 'Index': 0}, {'Processor': 'Image Normalization Preprocessing - Testing', 'Index': 1}]


```mermaid
---
config:
    layout: elk
---
graph LR
subgraph G0[Load MNIST - Load Supervised Features Block]
direction LR
X0[Load MNIST]
subgraph G0P[Ports]
direction TB
end
subgraph G0T[Terminals]
direction TB
XX0T0[X Train]
XX0T1[Y Train]
XX0T2[X Test]
XX0T3[Y Test]
end
X0 o--o XX0T0[X Train]
X0 o--o XX0T1[Y Train]
X0 o--o XX0T2[X Test]
X0 o--o XX0T3[Y Test]
end

```

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[Load MNIST - Load Supervised Features Block]
direction LR
subgraph GS0[Load MNIST System]
subgraph G1[Load MNIST Dataset - Load Supervised Dataset Block]
direction LR
X1[Load MNIST Dataset]
subgraph G1P[Ports]
direction TB
end
subgraph G1T[Terminals]
direction TB
XX1T0[X]
XX1T1[Y]
end
X1 o--o XX1T0[X]
X1 o--o XX1T1[Y]
end
subgraph G2[Test-Train Split - Cross Validation Split Block]
direction LR
X2[Test-Train Split]
subgraph G2P[Ports]
direction TB
XX2P0[X]
XX2P1[Y]
end
XX2P0[X] o--o X2
XX2P1[Y] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[X Train]
XX2T1[Y Train]
XX2T2[X Test]
XX2T3[Y Test]
end
X2 o--o XX2T0[X Train]
X2 o--o XX2T1[Y Train]
X2 o--o XX2T2[X Test]
X2 o--o XX2T3[Y Test]
end
subgraph G3[Image Normalization Preprocessing - Training - Training Data Preprocessing Block]
direction LR
X3[Image Normalization Preprocessing - Training]
subgraph G3P[Ports]
direction TB
XX3P0[X Train]
XX3P1[Y Train]
end
XX3P0[X Train] o--o X3
XX3P1[Y Train] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[X Train]
XX3T1[Y Train]
end
X3 o--o XX3T0[X Train]
X3 o--o XX3T1[Y Train]
end
subgraph G4[Image Normalization Preprocessing - Testing - Testing Data Preprocessing Block]
direction LR
X4[Image Normalization Preprocessing - Testing]
subgraph G4P[Ports]
direction TB
XX4P0[X Train]
XX4P1[Y Train]
XX4P2[X Test]
XX4P3[Y Test]
end
XX4P0[X Train] o--o X4
XX4P1[Y Train] o--o X4
XX4P2[X Test] o--o X4
XX4P3[Y Test] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[X Test]
XX4T1[Y Test]
end
X4 o--o XX4T0[X Test]
X4 o--o XX4T1[Y Test]
end
XX1T0[X] ---> XX2P0[X]
XX1T1[Y] ---> XX2P1[Y]
XX2T0[X Train] ---> XX3P0[X Train]
XX2T1[Y Train] ---> XX3P1[Y Train]
XX2T0[X Train] ---> XX4P0[X Train]
XX2T1[Y Train] ---> XX4P1[Y Train]
XX2T2[X Test] ---> XX4P2[X Test]
XX2T3[Y Test] ---> XX4P3[Y Test]
end
subgraph GC0P[Ports]
direction TB
end
subgraph GC0T[Terminals]
direction TB
X1T0[X Train]
X1T1[Y Train]
X1T2[X Test]
X1T3[Y Test]
end
XX3T0[X Train] --> X1T0[X Train]
XX3T1[Y Train] --> X1T1[Y Train]
XX4T0[X Test] --> X1T2[X Test]
XX4T1[Y Test] --> X1T3[Y Test]
end

```

In [ ]:



{
        "Description": "Conducts supervised learning using the defaults of the model",
        "ID": "Default Supervised Learning",
        "Name": "Default Supervised Learning",
        "Parent": "Supervised Learning",
        "Ports": [
          "Model",
          "X Train Array",
          "Y Train Array",
          "X Test Array",
          "Y Test Array"
        ],
        "Subsystem": {
          "Port Mappings": [
            {
              "Index": 0,
              "Processor": "Fit Supervised Model - Default"
            },
            {
              "Index": 1,
              "Processor": "Fit Supervised Model - Default"
            },
            {
              "Index": 2,
              "Processor": "Fit Supervised Model - Default"
            },
            {
              "Index": 1,
              "Processor": "Evaluate Supervised Model - Default"
            },
            {
              "Index": 2,
              "Processor": "Evaluate Supervised Model - Default"
            }
          ],
          "System ID": "Default Supervised Learning System",
          "Terminal Mappings": [
            {
              "Index": 0,
              "Processor": "No Post-processing"
            }
          ]
        },
        "Terminals": ["Evaluation Metrics"]
      },

({'Description': 'Conducts supervised learning using the defaults of the model',
  'ID': 'Default Supervised Learning',
  'Name': 'Default Supervised Learning',
  'Parent': 'Supervised Learning',
  'Ports': ['Model',
   'X Train Array',
   'Y Train Array',
   'X Test Array',
   'Y Test Array'],
  'Subsystem': {'Port Mappings': [{'Index': 0,
     'Processor': 'Fit Supervised Model - Default'},
    {'Index': 1, 'Processor': 'Fit Supervised Model - Default'},
    {'Index': 2, 'Processor': 'Fit Supervised Model - Default'},
    {'Index': 1, 'Processor': 'Evaluate Supervised Model - Default'},
    {'Index': 2, 'Processor': 'Evaluate Supervised Model - Default'}],
   'System ID': 'Default Supervised Learning System',
   'Terminal Mappings': [{'Index': 0, 'Processor': 'No Post-processing'}]},
  'Terminals': ['Evaluation Metrics']},)